# Install

In [1]:
%%bash
if test -d two-wai; then
  cd two-wai
  git pull
else
  git clone https://github.com/eriknomitch/two-wai.git
fi

Already up to date.


In [2]:
%cd two-wai

/notebooks/two-wai


In [3]:
%%bash
pip install -r requirements.txt > /dev/null

  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-avlixh02


# Banana

# Main

In [4]:
DEFAULT_MODEL_SIZE = "base"
DEFAULT_DELAY = 1.2

In [ ]:
import os
import gradio as gr
import whisper
from whisper import tokenizer
import time

current_size = DEFAULT_MODEL_SIZE
model = whisper.load_model(current_size)

def transcribe(audio, state={}, model_size=DEFAULT_MODEL_SIZE, delay=DEFAULT_DELAY):
    time.sleep(delay - 1)

    global current_size
    global model
    
    if model_size != current_size:
        current_size = model_size
        model = whisper.load_model(current_size)

    transcription = model.transcribe(
        audio,
        language='english'
    )
    state['transcription'] += transcription['text'] + " "
    
    print("-------------------------------------")
    print(f"ADDING: {transcription['text']}")
    print()
    print(f"ADDING: {state['transcription']}")

    return state['transcription'], state

100%|███████████████████████████████████████| 139M/139M [00:05<00:00, 27.5MiB/s]


In [ ]:
title = "two-way-speech"
description = "A demo of two-way-speech"

model_size = gr.Dropdown(label="Model size", choices=['base', 'tiny', 'small', 'medium', 'large'], value=DEFAULT_MODEL_SIZE)

delay_slider = gr.inputs.Slider(minimum=1, maximum=5, default=DEFAULT_DELAY, label="Rate of transcription")

transcription_tb = gr.Textbox(label="Transcription", lines=10, max_lines=500)

state = gr.State({"transcription": ""})

gr.Interface(
    fn=transcribe,
    inputs=[
        gr.Audio(source="microphone", type="filepath", streaming=True),
        state,
        model_size,
        delay_slider,
    ],
    outputs=[
        transcription_tb,
        state
    ],
    live=True,
    allow_flagging='never',
    title=title,
    description=description,
).launch(
    # enable_queue=True,
    debug=True,
    share=True
  )